In [2]:
import pandas as pd
import datetime as dt
from sklearn.linear_model import PoissonRegressor

In [3]:
df = pd.read_parquet('../data/processed/emprestimos_exemplares.parquet')

In [4]:
df_records = pd.read_parquet('../data/processed/students_records/records.parquet')

In [6]:
df_records['matricula_ou_siape'].unique()

array(['190001688', '199014489', '199735674', ..., '20200132517',
       '20200009344', '20200090284'], shape=(78576,), dtype=object)

In [6]:
df_records['tipo_vinculo_usuario'].unique()

array(['ALUNO DE GRADUAÇÃO'], dtype=object)

In [4]:
df.dtypes

id_emprestimo                  int64
codigo_barras_emprestimo      object
data_renovacao                object
data_emprestimo               object
data_devolucao                object
matricula_ou_siape            object
tipo_vinculo_usuario          object
id_exemplar                    int64
codigo_barras_exemplar        object
colecao                       object
biblioteca                    object
status_material               object
localizacao                    int64
CDU                         category
dtype: object

In [ ]:
df['data_emprestimo'] = pd.to_datetime(df['data_emprestimo'],format='ISO8601')

In [ ]:
data = df.merge(df_records)

data.head()

In [33]:
df['tipo_vinculo_usuario'].unique()

array(['ALUNO DE GRADUAÇÃO', 'ALUNO DE PÓS-GRADUAÇÃO', 'DOCENTE',
       'SERVIDOR TÉCNICO-ADMINISTRATIVO', 'DOCENTE EXTERNO',
       'ALUNO MÉDIO/TÉCNICO', 'USUÁRIO EXTERNO', 'OUTROS'], dtype=object)

In [4]:
graduacao_pos_2015 = df[df['tipo_vinculo_usuario'] == 'ALUNO DE GRADUAÇÃO']

graduacao_pos_2015 = graduacao_pos_2015.loc[:,['matricula_ou_siape','data_emprestimo']]

graduacao_pos_2015['data_emprestimo'] = pd.to_datetime(graduacao_pos_2015['data_emprestimo'])

graduacao_pos_2015 = graduacao_pos_2015[graduacao_pos_2015['data_emprestimo'].dt.year > 2015]

graduacao_pos_2015 = graduacao_pos_2015.reset_index(drop=True)

# graduacao_pos_2015.head()

In [5]:
cursos_selecionados = df_records[df_records['curso'].isin(['BIBLIOTECONOMIA','CIÊNCIAS SOCIAIS','COMUNICAÇÃO SOCIAL','DIREITO','FILOSOFIA','PEDAGOGIA'])]
# cursos_selecionados['curso'].unique()

In [6]:
usuarios_cursos = graduacao_pos_2015.merge(cursos_selecionados)

usuarios_cursos['ano_emprestimo'] = usuarios_cursos['data_emprestimo'].dt.year

emprestimos_cursos = usuarios_cursos.groupby(['ano_emprestimo', 'curso']).size().reset_index(name='quantidade')

# emprestimos_cursos.index = emprestimos_cursos['curso']
emprestimos_cursos = emprestimos_cursos.set_index('curso')

emprestimos_cursos

,ano_emprestimo,quantidade
curso,,
BIBLIOTECONOMIA,2016,7044
CIÊNCIAS SOCIAIS,2016,6984
COMUNICAÇÃO SOCIAL,2016,7973
DIREITO,2016,7363
FILOSOFIA,2016,7624
PEDAGOGIA,2016,7174
BIBLIOTECONOMIA,2017,7307
CIÊNCIAS SOCIAIS,2017,6926
COMUNICAÇÃO SOCIAL,2017,8253


In [ ]:
emprestimos_cursos['ano_emprestimo'].unique()

In [ ]:
emprestimos_cursos.index.unique()

In [7]:
previsoes = []
# total_2023 = 0

# Iterar sobre cada curso (excluindo o total)
for curso in emprestimos_cursos.index.unique():
    serie_curso = emprestimos_cursos.loc[[curso], ['ano_emprestimo', 'quantidade']]
    
    X = serie_curso[['ano_emprestimo']]
    y = serie_curso['quantidade']
    
    # Treinar Prophet
    model = PoissonRegressor(alpha=0.0, max_iter=1000)
    model.fit(X, y)
    
    # Guardar resultado
    valor_2023 = int(model.predict([[2023]])[0])
    previsoes.append({'curso': curso, '2023':  max(0, valor_2023)})
    
    # total_2023 +=  max(0, valor_2023)

# previsoes.append({'curso': 'TOTAL', '2023': total_2023})

# Transformar previsões em DataFrame
df_previsoes = pd.DataFrame(previsoes).set_index('curso')
df_previsoes

f:\pedro\Documents\dev\lab402\envs\py_313_data\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but PoissonRegressor was fitted with feature names
  warnings.warn(
f:\pedro\Documents\dev\lab402\envs\py_313_data\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but PoissonRegressor was fitted with feature names
  warnings.warn(
f:\pedro\Documents\dev\lab402\envs\py_313_data\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but PoissonRegressor was fitted with feature names
  warnings.warn(
f:\pedro\Documents\dev\lab402\envs\py_313_data\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but PoissonRegressor was fitted with feature names
  warnings.warn(
f:\pedro\Documents\dev\lab402\envs\py_313_data\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have vali

,2023
curso,
BIBLIOTECONOMIA,814
CIÊNCIAS SOCIAIS,803
COMUNICAÇÃO SOCIAL,668
DIREITO,1242
FILOSOFIA,773
PEDAGOGIA,876


In [ ]:
emprestimos_cursos.index

In [25]:
df_previsoes.index

Index(['BIBLIOTECONOMIA', 'CIÊNCIAS SOCIAIS', 'COMUNICAÇÃO SOCIAL', 'DIREITO',
       'FILOSOFIA', 'PEDAGOGIA'],
      dtype='object', name='curso')

In [12]:
emprestimos_cursos.dtypes

ano_emprestimo    int32
quantidade        int64
dtype: object

In [22]:
df_previsoes.dtypes

ano_emprestimo    int32
quantidade        int64
dtype: object

In [9]:
df_previsoes = df_previsoes.reset_index().melt(id_vars='curso', var_name='ano_emprestimo',value_name='quantidade')
# df_previsoes = df_previsoes.reset_index()

In [13]:
df_previsoes = df_previsoes.set_index('curso')

In [21]:
df_previsoes['ano_emprestimo'] = df_previsoes['ano_emprestimo'].astype('int32')

In [14]:
df_previsoes

,ano_emprestimo,quantidade
curso,,
BIBLIOTECONOMIA,2023,814
CIÊNCIAS SOCIAIS,2023,803
COMUNICAÇÃO SOCIAL,2023,668
DIREITO,2023,1242
FILOSOFIA,2023,773
PEDAGOGIA,2023,876


In [23]:
df_emprestimos = emprestimos_cursos.merge(df_previsoes)
df_emprestimos

,ano_emprestimo,quantidade


In [26]:
df_emprestimos = pd.concat(
    [emprestimos_cursos, df_previsoes]
)

df_emprestimos

,ano_emprestimo,quantidade
curso,,
BIBLIOTECONOMIA,2016,7044
CIÊNCIAS SOCIAIS,2016,6984
COMUNICAÇÃO SOCIAL,2016,7973
DIREITO,2016,7363
FILOSOFIA,2016,7624
PEDAGOGIA,2016,7174
BIBLIOTECONOMIA,2017,7307
CIÊNCIAS SOCIAIS,2017,6926
COMUNICAÇÃO SOCIAL,2017,8253


In [30]:
df_emprestimos = df_emprestimos.reset_index()

In [31]:
emprestimos_cursos_pivot = df_emprestimos.pivot_table(
    index='curso',
    columns='ano_emprestimo',
    values='quantidade',
    fill_value = 0,
    aggfunc= 'sum',
    margins = True,
    margins_name = 'TOTAL',
)

emprestimos_cursos_pivot

ano_emprestimo,2016,2017,2018,2019,2020,2021,2022,2023,TOTAL
curso,,,,,,,,,
BIBLIOTECONOMIA,7044,7307,6567,6320,964,69,1175,814,30260
CIÊNCIAS SOCIAIS,6984,6926,6680,6738,909,76,951,803,30067
COMUNICAÇÃO SOCIAL,7973,8253,6953,5885,807,66,942,668,31547
DIREITO,7363,7885,8466,9203,1352,99,1331,1242,36941
FILOSOFIA,7624,7427,6814,6248,893,101,1136,773,31016
PEDAGOGIA,7174,7464,7094,6783,957,123,1166,876,31637
TOTAL,44162,45262,42574,41177,5882,534,6701,5176,191468


In [ ]:
emprestimos_cursos_pivot.index

In [ ]:
df_previsoes

In [ ]:
emprestimos_cursos_pivot